In [1]:
import os
import cv2
from keras.preprocessing import image
import matplotlib.pyplot as plt 
import numpy as np
from keras.utils.np_utils import to_categorical
import random,shutil
from keras.models import Sequential
from keras.layers import Dropout,Conv2D,Flatten,Dense, MaxPooling2D, BatchNormalization
from keras.models import load_model

In [2]:
def generator(dir, gen=image.ImageDataGenerator(rescale=1./255), shuffle=True,batch_size=1,target_size=(24,24),class_mode='categorical' ):

    return gen.flow_from_directory(dir,batch_size=batch_size,shuffle=shuffle,color_mode='grayscale',class_mode=class_mode,target_size=target_size)

In [3]:
BS= 32
TS=(24,24)
train_batch= generator('data/train',shuffle=True, batch_size=BS,target_size=TS)
valid_batch= generator('data/valid',shuffle=True, batch_size=BS,target_size=TS)
SPE= len(train_batch.classes)//BS
VS = len(valid_batch.classes)//BS
print(SPE,VS)

Found 1234 images belonging to 2 classes.
Found 218 images belonging to 2 classes.
38 6


In [4]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
#32 convolution filters used each of size 3x3
#again
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1,1)),

#64 convolution filters used each of size 3x3
#choose the best features via pooling
    
#randomly turn neurons on and off to improve convergence
    Dropout(0.25),
#flatten since too many dimensions, we only want a classification output
    Flatten(),
#fully connected to get all relevant data
    Dense(128, activation='relu'),
#one more dropout for convergence' sake :) 
    Dropout(0.5),
#output a softmax to squash the matrix into output probabilities
    Dense(2, activation='softmax')
])


In [7]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 22, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 18, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 18, 18, 64)        0

In [10]:
model.fit_generator(train_batch, validation_data=valid_batch,epochs=15,steps_per_epoch=SPE ,validation_steps=VS)

H:\apps files\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
38/38 [==============================] - 23s 459ms/step - loss: 0.5464 - accuracy: 0.7029 - val_loss: 0.2090 - val_accuracy: 0.9115
Epoch 2/15
38/38 [==============================] - 9s 230ms/step - loss: 0.1934 - accuracy: 0.9318 - val_loss: 0.1254 - val_accuracy: 0.9531
Epoch 3/15
38/38 [==============================] - 6s 171ms/step - loss: 0.1251 - accuracy: 0.9559 - val_loss: 0.1269 - val_accuracy: 0.9427
Epoch 4/15
38/38 [==============================] - 6s 170ms/step - loss: 0.0805 - accuracy: 0.9717 - val_loss: 0.1000 - val_accuracy: 0.9531
Epoch 5/15
38/38 [==============================] - 9s 235ms/step - loss: 0.0627 - accuracy: 0.9773 - val_loss: 0.1093 - val_accuracy: 0.9583
Epoch 6/15
38/38 [==============================] - 9s 229ms/step - loss: 0.0492 - accuracy: 0.9782 - val_loss: 0.1269 - val_accuracy: 0.9583
Epoch 7/15
38/38 [==============================] - 8s 209ms/step - loss: 0.0637 - accuracy: 0.9809 - val_loss: 0.0997 - val_accuracy: 0.9583
Epoch

In [11]:
model.save('models/cnnCat2.h5', overwrite=True)